In [10]:
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy import stats
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from datetime import datetime
import statsmodels.api as sm
from pandas.plotting import lag_plot
from pmdarima.arima import auto_arima
import warnings
warnings.filterwarnings("ignore")
import pmdarima as pm
import os
from statsmodels.tsa.arima.model import ARIMA 
directory = "./data1/"
list_rs2 = []
data1 = pd.DataFrame()
Months12 = pd.DataFrame()
df_results1 = pd.DataFrame(columns=["Name", "RMSE"])
for filename in os.listdir(directory):
    try:
        if os.path.isfile(os.path.join(directory, filename)):
            print("ชื่อไฟล์:", filename)
            data1 = pd.read_csv(f"./data1/{filename}",parse_dates=['nav_date'])
            data1['nav_date'] = pd.to_datetime(data1['nav_date'])
            monthly_avg = data1.set_index('nav_date').resample('M').mean()
            df = pd.DataFrame(monthly_avg["last_val"])
            df = pd.DataFrame(df["last_val"])
            df = df.dropna()
            #df_RS2 = pd.concat([df_RS2, df_Rs2_temp])
            #df_RS2 = df_RS2._append(df_Rs2_temp)
            num = 6 #ตัดข้อมูลย้อนหลังกลับไป 6 เดือนเพื่อที่จะทำการวัด rmse 
            train_size = int(len(df)-num)
            test_size = len(df) - train_size
            train_data = df.iloc[:train_size]
            test_data = df.iloc[train_size:]
            scaler = MinMaxScaler(feature_range = (0,1))
            df_scaled = scaler.fit_transform(df)
            train_scaled = df_scaled[:train_size]
            test_scaled = df_scaled[train_size:]
            auto_arima = pm.auto_arima(train_scaled, stepwise=False, seasonal=True)
            forecastnum = 12
            forecast_test_auto = auto_arima.predict(n_periods=len(test_scaled)+forecastnum)
            forcastnew_data = scaler.inverse_transform(forecast_test_auto.reshape(-1,1))
            forcastnew_data_back6months = forcastnew_data[:-forecastnum]
            forcastnew_data_1year = forcastnew_data[-forecastnum:] 
            RMSE_error = mean_squared_error(test_data, forcastnew_data_back6months)
            print('Testing Mean Squared Error is {}'.format(RMSE_error))
            from datetime import datetime, timedelta
            import calendar
            current_date = datetime.now()
            current_month = datetime.now().month
            months = [(calendar.month_name[(current_month + i - 1) % 12 + 1]) for i in range(12)]
            Months12 = pd.DataFrame({'Month': months})
            Months12["invesment"] = 10000 #invesment
            #Months12['expense'] = np.random.randint(1000, 5001, size=len(Months12))
            #Months12['invesment'] = np.random.randint(1000, 5001, size=len(Months12))
            #Months12['remaining'] = Months12["income"] - Months12['expense'] - Months12['invesment']
            Months12["Stockprice_predicted(Months)"] = forcastnew_data_1year
            Months12['units'] = Months12["invesment"]/Months12["Stockprice_predicted(Months)"]
            Months12["sum_units"] = Months12['units'].cumsum()
            Months12["Sum_Values_Monthly"] = Months12["sum_units"] * Months12["Stockprice_predicted(Months)"]
            Months12["income_sum"] = Months12["invesment"].cumsum()
            Months12["Growth_rate_percent"] = ((Months12["Sum_Values_Monthly"] - Months12["income_sum"])/Months12["income_sum"])*100
            avg_growthrate = Months12["Growth_rate_percent"].mean()
            print('growth rate is {}'.format(avg_growthrate))
            df_results1 = df_results1._append({"Name": filename, "RMSE": RMSE_error,"Avg_Growth_rate_1Y":avg_growthrate}, ignore_index=True)
    except Exception as e:
        print(f"Error or Empty {filename}: {e}")
        continue

ชื่อไฟล์: M0003_2554_NON.csv
Testing Mean Squared Error is 0.0005324095000610366
growth rate is 0.7447096142705698
ชื่อไฟล์: M0003_2562_NON.csv
Testing Mean Squared Error is 0.012840034832668552
growth rate is -0.00427308161435628
ชื่อไฟล์: M0005_2558_SSF.csv
Testing Mean Squared Error is 1.8194397322830005
growth rate is 0.1257432137102079
ชื่อไฟล์: M0005_2565_SSF.csv
Testing Mean Squared Error is 0.11897736052641923
growth rate is -6.736997790910581e-16
ชื่อไฟล์: M0017_2565_SSF.csv
Testing Mean Squared Error is 1.1160928825305003
growth rate is 0.0
ชื่อไฟล์: M0059_2564_SSF.csv
Testing Mean Squared Error is 0.9152004801841684
growth rate is -12.416553271607441
ชื่อไฟล์: M0074_2544_RMF.csv
Testing Mean Squared Error is 0.0017807514270278331
growth rate is 0.33036490394310997
ชื่อไฟล์: M0080_2544_RMF.csv
Testing Mean Squared Error is 2.2954077391601393
growth rate is 0.6774414041154376
ชื่อไฟล์: M0090_2561_SSF.csv
Testing Mean Squared Error is 0.051348906518079634
growth rate is -0.3255

In [11]:
df_results1

,Name,RMSE,Avg_Growth_rate_1Y
0,M0003_2554_NON.csv,0.000532410,0.744709614
1,M0003_2562_NON.csv,0.012840035,-0.004273082
2,M0005_2558_SSF.csv,1.819439732,0.125743214
3,M0005_2565_SSF.csv,0.118977361,-0.000000000
4,M0017_2565_SSF.csv,1.116092883,0.000000000
5,M0059_2564_SSF.csv,0.915200480,-12.416553272
6,M0074_2544_RMF.csv,0.001780751,0.330364904
7,M0080_2544_RMF.csv,2.295407739,0.677441404
8,M0090_2561_SSF.csv,0.051348907,-0.325562520
9,M0110_2564_SSF.csv,2.913945091,-5.972219914


In [3]:
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy import stats
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from datetime import datetime
import statsmodels.api as sm
from pandas.plotting import lag_plot
from pmdarima.arima import auto_arima
import warnings
warnings.filterwarnings("ignore")
import pmdarima as pm
import os
from statsmodels.tsa.arima.model import ARIMA 
directory = "./data2/"
list_rs2 = []
data2 = pd.DataFrame()
Months12 = pd.DataFrame()
df_results2 = pd.DataFrame(columns=["Name", "RMSE"])
for filename in os.listdir(directory):
    # Check if the file is a regular file
    try:
        if os.path.isfile(os.path.join(directory, filename)):
            print("ชื่อไฟล์:", filename)
            data2 = pd.read_csv(f"./data2/{filename}",parse_dates=['nav_date'])
            data2['nav_date'] = pd.to_datetime(data2['nav_date'])
            monthly_avg = data2.set_index('nav_date').resample('M').mean()
            df = pd.DataFrame(monthly_avg["last_val"])
            df = pd.DataFrame(df["last_val"])
            df = df.dropna()
            #df_RS2 = pd.concat([df_RS2, df_Rs2_temp])
            #df_RS2 = df_RS2._append(df_Rs2_temp)
            num = 6
            train_size = int(len(df)-num)
            test_size = len(df) - train_size
            train_data = df.iloc[:train_size]
            test_data = df.iloc[train_size:]
            scaler = MinMaxScaler(feature_range = (0,1))
            df_scaled = scaler.fit_transform(df)
            train_scaled = df_scaled[:train_size]
            test_scaled = df_scaled[train_size:]
            auto_arima = pm.auto_arima(train_scaled, stepwise=False, seasonal=True)
            forecastnum = 12
            forecast_test_auto = auto_arima.predict(n_periods=len(test_scaled)+forecastnum)
            forcastnew_data = scaler.inverse_transform(forecast_test_auto.reshape(-1,1))
            forcastnew_data_back6months = forcastnew_data[:-forecastnum]
            forcastnew_data_1year = forcastnew_data[-forecastnum:] 
            RMSE_error = mean_squared_error(test_data, forcastnew_data_back6months)
            print('Testing Mean Squared Error is {}'.format(RMSE_error))
            from datetime import datetime, timedelta
            import calendar
            current_date = datetime.now()
            current_month = datetime.now().month
            months = [(calendar.month_name[(current_month + i - 1) % 12 + 1]) for i in range(12)]
            Months12 = pd.DataFrame({'Month': months})
            Months12["invesment"] = 10000 #invesment
            #Months12['expense'] = np.random.randint(1000, 5001, size=len(Months12))
            #Months12['invesment'] = np.random.randint(1000, 5001, size=len(Months12))
            #Months12['remaining'] = Months12["income"] - Months12['expense'] - Months12['invesment']
            Months12["Stockprice_predicted(Months)"] = forcastnew_data_1year
            Months12['units'] = Months12["invesment"]/Months12["Stockprice_predicted(Months)"]
            Months12["sum_units"] = Months12['units'].cumsum()
            Months12["Sum_Values_Monthly"] = Months12["sum_units"] * Months12["Stockprice_predicted(Months)"]
            Months12["income_sum"] = Months12["invesment"].cumsum()
            Months12["Growth_rate_percent"] = ((Months12["Sum_Values_Monthly"] - Months12["income_sum"])/Months12["income_sum"])*100
            avg_growthrate = Months12["Growth_rate_percent"].mean()
            print('growth rate is {}'.format(avg_growthrate))
            df_results2 = df_results2._append({"Name": filename, "RMSE": RMSE_error,"Avg_Growth_rate_1Y":avg_growthrate}, ignore_index=True)
    except Exception as e:
        print(f"Error or Empty {filename}: {e}")
        continue

C:\Users\User\AppData\Local\Temp\ipykernel_59192\818315476.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


ชื่อไฟล์: M0452_2546_SSF.csv
Testing Mean Squared Error is 0.03617039860372752
growth rate is 0.2624833610991888
ชื่อไฟล์: M0464_2548_SSF.csv
Testing Mean Squared Error is 0.22259966782932963
growth rate is 0.8473385189124666
ชื่อไฟล์: M0465_2564_SSF.csv
Testing Mean Squared Error is 0.8773494687947659
growth rate is -0.3782026289557939
ชื่อไฟล์: M0466_2564_SSF.csv
Testing Mean Squared Error is 1.6023195843365594
growth rate is 0.5469892128643639
ชื่อไฟล์: M0489_2553_SSF.csv
Testing Mean Squared Error is 0.004736858063794242
growth rate is -0.007557818656583615
ชื่อไฟล์: M0513_2564_SSF.csv
Error or Empty M0513_2564_SSF.csv: too many indices for array: array is 0-dimensional, but 1 were indexed
ชื่อไฟล์: M0515_2565_SSF.csv
Error or Empty M0515_2565_SSF.csv: Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required by MinMaxScaler.
ชื่อไฟล์: M0536_2563_SSF.csv
Testing Mean Squared Error is 2.716630516032557
growth rate is 3.766449957495419
ชื่อไฟล์: M0539_2564_SSF.csv


In [7]:
pd.set_option('display.float_format', lambda x: '%.9f' % x)

In [9]:
df_results2

,Name,RMSE,Avg_Growth_rate_1Y
0,M0452_2546_SSF.csv,0.036170399,0.262483361
1,M0464_2548_SSF.csv,0.222599668,0.847338519
2,M0465_2564_SSF.csv,0.877349469,-0.378202629
3,M0466_2564_SSF.csv,1.602319584,0.546989213
4,M0489_2553_SSF.csv,0.004736858,-0.007557819
5,M0536_2563_SSF.csv,2.716630516,3.766449957
6,M0539_2564_SSF.csv,4.400805086,0.000000000
7,M0548_2564_RMF.csv,5.266964486,-30.340502258
8,M0550_2560_SSF.csv,0.158698574,-0.165117026
9,M0593_2558_RMF.csv,0.046910856,1.035475373


In [12]:
df_results_sum = df_results2._append(df_results1)

In [13]:
df_results_sum = df_results_sum.reset_index(drop=True)

In [14]:
df_results_sum

,Name,RMSE,Avg_Growth_rate_1Y
0,M0452_2546_SSF.csv,0.036170399,0.262483361
1,M0464_2548_SSF.csv,0.222599668,0.847338519
2,M0465_2564_SSF.csv,0.877349469,-0.378202629
3,M0466_2564_SSF.csv,1.602319584,0.546989213
4,M0489_2553_SSF.csv,0.004736858,-0.007557819
...,...,...,...
58,M0348_2561_SSF.csv,0.124058932,-0.213499190
59,M0386_2564_SSF.csv,0.260240527,0.104689333
60,M0396_2564_SSF.csv,0.126669665,1.156298406
61,M0420_2564_SSF.csv,7.459112461,4.741494733


In [15]:
df_results_sum.to_csv("rmse_growth_all_fund.csv")